## Desagregacion Forecast Windows

Este notebook se va a encargar de:

- Generar un archivo csv para el datatraining
- Generar un archivo csv para el datatesting
- Generar el archivo final para cargar al SQL

In [1]:
import modulo_conn_sql as mcq
import numpy as np
import pandas as pd 
import datetime 
from pandas.tseries.offsets import MonthEnd
from pandas.tseries.offsets import MonthBegin

import sqlalchemy as sa
import urllib

from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from xgboost import XGBRegressor

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from livelossplot import PlotLossesKeras
import keras.optimizers as opts

from pickle import dump
from pickle import load
from keras.models import load_model

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

#Query BD SQL-Server Cemex
def querySQL(query, parametros):
    #Conectar con base sql y ejecutar consulta
    cursor = conectarSQL()
    try:
        cursor.execute(query, parametros)
        #obtener nombre de columnas
        names = [ x[0] for x in cursor.description]
        
        #Reunir todos los resultado en rows
        rows = cursor.fetchall()
        resultadoSQL = []
            
        #Hacer un array con los resultados
        while rows:
            resultadoSQL.append(rows)
            if cursor.nextset():
                rows = cursor.fetchall()
            else:
                rows = None
                
        #Redimensionar el array para que quede en dos dimensiones
        resultadoSQL = np.array(resultadoSQL)
        resultadoSQL = np.reshape(resultadoSQL, (resultadoSQL.shape[1], resultadoSQL.shape[2]) )
    finally:
            if cursor is not None:
                cursor.close()
    return pd.DataFrame(resultadoSQL, columns = names)

#SQL Methods to get operation data
def conectarSQL():
    conn = mcq.ConexionSQL()
    cursor = conn.getCursor()
    return cursor

# dataset(pandas dataframe): base de datos con el historico
# array_group_top(array) : array de la jerarquia mas ALTA con el nombre de las columnas del dataset por el cual se quiere agrupar las proporciones
# array_group_bottom(array) : array de la jerarquia mas BAJA con el nombre de las columnas del dataset por el cual se quiere agrupar las proporciones
# medida_target( string ) : nombre de la columna que contiene los datos objetivo de la proporcion
# group_target(array) : array de nombre de columnas con las cuales queremos la proporcion final
# name_proportion(string) : etiqueta de la dimension a la cual le estamos calculando la proporcion
    
def historical_proportion( dataset, array_group_top, array_group_bottom, medida_target, group_target, name_proportion  ):

    promedio_group_top = dataset.groupby(array_group_top)[medida_target].mean().reset_index()
    promedio_group_bottom = dataset.groupby(array_group_bottom)[medida_target].mean().reset_index()    
    proportion  = pd.merge(promedio_group_bottom, promedio_group_top, how = 'inner', left_on = array_group_top, right_on = array_group_top )
    proportion['proportion'] = proportion[medida_target + '_x'] / proportion[medida_target + '_y']
    proportion = proportion.groupby(group_target)['proportion'].mean().reset_index()
    proportion.rename(columns={'proportion':'proportion_' + name_proportion}, inplace = True)
    
    return proportion 




# DATA TRAINING

In [84]:
def calculate_proportion(df_total, months, denominador, numerador, columna_target, grupo_final, name_column_return):
    #proporcion de Semana
    df_proportion_semana = pd.DataFrame()

    for i in df_total['year_month'].unique():

        mes =  df_total[df_total['year_month'] == i]['Mes'].unique()[0]
        año =  df_total[df_total['year_month'] == i]['Año'].unique()[0]

        df_param = df_total[(df_total['FechaEntrega'] >= datetime.datetime(año, mes , 1) - MonthBegin(months)) &  (df_total['FechaEntrega'] < datetime.datetime(año, mes , 1))]
        if len(df_param) > 0:
            df_proportion_semana_detalle = historical_proportion(df_param, denominador, numerador, columna_target, grupo_final, name_column_return)
            df_proportion_semana_detalle['Año'] = año
            df_proportion_semana_detalle['Mes'] = mes
            if len(df_proportion_semana_detalle) == 0:
                df_proportion_semana = df_proportion_semana_detalle
            else:
                df_proportion_semana = pd.concat([df_proportion_semana, df_proportion_semana_detalle])

    df_proportion_semana = df_proportion_semana.fillna(0)
    
    return df_proportion_semana.reset_index(drop=True)
    
def media_diaria(df_total, months):
    
    media_diaria_total = pd.DataFrame()

    for i in df_total['year_month'].unique():

        mes =  df_total[df_total['year_month'] == i]['Mes'].unique()[0]
        año =  df_total[df_total['year_month'] == i]['Año'].unique()[0]

        df_param = df_total[(
            df_total['FechaEntrega'] >= datetime.datetime(año, mes , 1) - MonthBegin(months)) &  
            (df_total['FechaEntrega'] < datetime.datetime(año, mes , 1))]

        media_diaria = df_param.groupby(
            [
            'Año', 
            'Mes', 
            'Planta'
            ]
        ).agg(
            {
            'totalEntregado': 'sum', 
            'DiasOperativos':'max' }
        ).reset_index()

        media_diaria['media'] = media_diaria['totalEntregado'] /  media_diaria['DiasOperativos']

        media_diaria = media_diaria.groupby(['Planta'])['media'].mean().reset_index()
        media_diaria['Año'] = año
        media_diaria['Mes'] = mes
        media_diaria.rename(columns={'media':'media'+str(months)}, inplace = True)
        
        if len(media_diaria_total) == 0:
            media_diaria_total = media_diaria
        else:
            media_diaria_total = pd.concat([media_diaria_total, media_diaria])
        
    return media_diaria_total.reset_index(drop=True)


def volumen_mes(df_total):
    
    volumen_mes = pd.DataFrame()
        
    volumen_mes = df_total.groupby(
        [
        'Año', 
        'Mes'
        ]
    ).agg(
        {
        'totalEntregado': 'sum' 
        }
    ).reset_index()
    
    volumen_mes.rename(columns={'totalEntregado':'volumen_ciudad'}, inplace = True)

    return volumen_mes
    

In [85]:
pais = 'Colombia'
inicioHistoria = '2014-01-01'
finHistoria = '2022-05-31'

#Consulta historico de volumen despachado diario por planta

despachosSQL = querySQL("{CALL SCAC_AP8_BaseForecast (?,?,?)}", (pais, inicioHistoria, finHistoria ) )
despachosSQL['year_month'] = despachosSQL.FechaEntrega.dt.to_period('M')
despachosSQL['totalEntregado'] = despachosSQL['totalEntregado'].astype(float)

#agrego informacion geografica        
nombre_cluster = querySQL( "SELECT Pais, Centro, Ciudad_Cluster as Ciudad, [Desc Cluster] as Cluster FROM SCAC_AT1_NombreCluster where Pais = ?" , (pais) )

despachosSQL = pd.merge(despachosSQL, nombre_cluster, left_on ='Planta', right_on='Centro', how = 'left')

proportion_week_1 = calculate_proportion(despachosSQL, 
                                         1, 
                                         ['Año', 'Mes', 'Planta'], 
                                         ['Año', 'Mes', 'Planta', 'Semana_Relativa'], 
                                         'totalEntregado', 
                                         ['Planta', 'Semana_Relativa'],
                                         'semana' + str(1))

proportion_week_2 = calculate_proportion(despachosSQL, 
                                         2, 
                                         ['Año', 'Mes', 'Planta'], 
                                         ['Año', 'Mes', 'Planta', 'Semana_Relativa'], 
                                         'totalEntregado', 
                                         ['Planta', 'Semana_Relativa'],
                                         'semana' + str(2))

proportion_week_3 = calculate_proportion(despachosSQL, 
                                         3, 
                                         ['Año', 'Mes', 'Planta'], 
                                         ['Año', 'Mes', 'Planta', 'Semana_Relativa'], 
                                         'totalEntregado', 
                                         ['Planta', 'Semana_Relativa'],
                                         'semana' + str(3))

proportion_week_6 = calculate_proportion(despachosSQL, 
                                         6, 
                                         ['Año', 'Mes', 'Planta'], 
                                         ['Año', 'Mes', 'Planta', 'Semana_Relativa'], 
                                         'totalEntregado', 
                                         ['Planta', 'Semana_Relativa'],
                                         'semana' + str(6))

proportion_weekday_1 = calculate_proportion(despachosSQL, 
                                         1, 
                                         ['Año', 'Mes', 'Planta'], 
                                         ['Año', 'Mes', 'Planta', 'DiaSemana'], 
                                         'totalEntregado', 
                                         ['Planta', 'DiaSemana'],
                                         'dia_semana' + str(1))

proportion_weekday_2 = calculate_proportion(despachosSQL, 
                                         2, 
                                         ['Año', 'Mes', 'Planta'], 
                                         ['Año', 'Mes', 'Planta', 'DiaSemana'], 
                                         'totalEntregado', 
                                         ['Planta', 'DiaSemana'],
                                         'dia_semana' + str(2))

proportion_weekday_3 = calculate_proportion(despachosSQL, 
                                         3, 
                                         ['Año', 'Mes', 'Planta'], 
                                         ['Año', 'Mes', 'Planta', 'DiaSemana'], 
                                         'totalEntregado', 
                                         ['Planta', 'DiaSemana'],
                                         'dia_semana' + str(3))

proportion_weekday_6 = calculate_proportion(despachosSQL, 
                                         6, 
                                         ['Año', 'Mes', 'Planta'], 
                                         ['Año', 'Mes', 'Planta', 'DiaSemana'], 
                                         'totalEntregado', 
                                         ['Planta', 'DiaSemana'],
                                         'dia_semana' + str(6))

media_diaria1 = media_diaria(despachosSQL, 1)
media_diaria2 = media_diaria(despachosSQL, 2)
media_diaria3 = media_diaria(despachosSQL, 3)
media_diaria6 = media_diaria(despachosSQL, 6)


volumen_ciudad_mes = volumen_mes(despachosSQL)

df = pd.merge(despachosSQL, proportion_week_1, on=['Año', 'Mes', 'Planta', 'Semana_Relativa'], how='left')
df = pd.merge(df, proportion_week_2, on=['Año', 'Mes', 'Planta', 'Semana_Relativa'], how='left')
df = pd.merge(df, proportion_week_3, on=['Año', 'Mes', 'Planta', 'Semana_Relativa'], how='left')
df = pd.merge(df, proportion_week_6, on=['Año', 'Mes', 'Planta', 'Semana_Relativa'], how='left')

df = pd.merge(df, proportion_weekday_1, on=['Año', 'Mes', 'Planta', 'DiaSemana'], how='left')
df = pd.merge(df, proportion_weekday_2, on=['Año', 'Mes', 'Planta', 'DiaSemana'], how='left')
df = pd.merge(df, proportion_weekday_3, on=['Año', 'Mes', 'Planta', 'DiaSemana'], how='left')
df = pd.merge(df, proportion_weekday_6, on=['Año', 'Mes', 'Planta', 'DiaSemana'], how='left')

df = pd.merge(df, media_diaria1, on=['Año', 'Mes', 'Planta'], how='left')
df = pd.merge(df, media_diaria2, on=['Año', 'Mes', 'Planta'], how='left')
df = pd.merge(df, media_diaria3, on=['Año', 'Mes', 'Planta'], how='left')
df = pd.merge(df, media_diaria6, on=['Año', 'Mes', 'Planta'], how='left')

#df= pd.merge(df, volumen_ciudad_mes, on=['Año','Mes', 'Ciudad'], how='left')
df= pd.merge(df, volumen_ciudad_mes, on=['Año','Mes'], how='left')

df['PlantaCentral'] = np.select(
    [
        df['TipoPlanta'] == 'Central'
    ],
    [
        1
    ],default=0)

df = df.fillna(0)

columnas = ['Año', 'Mes', 'Planta','Semana_Relativa', 'DiaSemana', 
            'proportion_semana1', 'proportion_semana2',
            'proportion_semana3', 'proportion_semana6', 
            'proportion_dia_semana1','proportion_dia_semana2', 
            'proportion_dia_semana3', 'proportion_dia_semana6', 
            'media1', 'media2', 'media3', 'media6',
            'PlantaCentral', 'volumen_ciudad', 'totalEntregado']

df = df[columnas]

df.to_csv('../datos/df_datatraining.csv')

# DATA TESTING (Para prediccion)

In [89]:
def calculate_proportion_testing(df_total, months, año, mes, denominador, numerador, columna_target, grupo_final, name_column_return):
    
    #proporcion de Semana
    df_proportion_semana = pd.DataFrame()

    df_param = df_total[(df_total['FechaEntrega'] >= datetime.datetime(año, mes , 1) - MonthBegin(months)) &  (df_total['FechaEntrega'] < datetime.datetime(año, mes , 1))]
    if len(df_param) > 0:
        df_proportion_semana_detalle = historical_proportion(df_param, denominador, numerador, columna_target, grupo_final, name_column_return)
        df_proportion_semana_detalle['Año'] = año
        df_proportion_semana_detalle['Mes'] = mes
        
        
        df_proportion_semana = df_proportion_semana_detalle.copy()
        

    df_proportion_semana = df_proportion_semana.fillna(0)
    
    return df_proportion_semana.reset_index(drop=True)

def media_diaria_testing(df_total, months, año, mes):
    
    media_diaria_total = pd.DataFrame()

    df_param = df_total[(
        df_total['FechaEntrega'] >= datetime.datetime(año, mes , 1) - MonthBegin(months)) &  
        (df_total['FechaEntrega'] < datetime.datetime(año, mes , 1))]

    media_diaria = df_param.groupby(
        [
        'Año', 
        'Mes', 
        'Planta'
        ]
    ).agg(
        {
        'totalEntregado': 'sum', 
        'DiasOperativos':'max' }
    ).reset_index()

    media_diaria['media'] = media_diaria['totalEntregado'] /  media_diaria['DiasOperativos']

    media_diaria = media_diaria.groupby(['Planta'])['media'].mean().reset_index()
    media_diaria['Año'] = año
    media_diaria['Mes'] = mes
    media_diaria.rename(columns={'media':'media'+str(months)}, inplace = True)

    media_diaria_total = media_diaria.copy()
        
    return media_diaria_total.reset_index(drop=True)

def volumen_mes_testing(df_total, año, mes, fecha_ref):
    
    """    
    volumen_mes = pd.DataFrame()
        
    volumen_mes = df_total[df_total['FechaEntrega'] >=  fecha_ref - datetime.timedelta(30) ].groupby(
        [ 
        'Año', 
        'Mes'
        ]
    ).agg(
        {
        'totalEntregado': 'sum' 
        }
    ).reset_index()
    
    volumen_mes['Año'] = año
    volumen_mes['Mes'] = mes
    
    volumen_mes.rename(columns={'totalEntregado':'volumen_ciudad'}, inplace = True)
    """

    return df_total['totalEntregado'].sum()

In [90]:
año_target = 2022
mes_target = 6
pais = 'Colombia'

#finHistoria = datetime.datetime.today()
finHistoria = datetime.datetime(2022,5,31)
inicioHistoria = datetime.datetime(año_target, mes_target, 1 ) - MonthBegin(7)

#Volumen pre-establecido por ciudad
volumen_testing_ciudad = pd.read_excel('../DatosAbsorcionPlantas/ColombiaCiudad.xlsx')

df_calendario = querySQL( 
    "SELECT * FROM SCAC_AT3_DiasHabilesFuente WHERE pais = ? and [Fecha de entrega]  between ? and ?" , 
    (pais, datetime.datetime(año_target, mes_target, 1).strftime("%Y-%m-%d"), 
     (datetime.datetime(año_target, mes_target, 1) + MonthEnd(1)).strftime("%Y-%m-%d") )
)

df_calendario.rename(
    columns={'Fecha de entrega':'FechaEntrega'}, 
    inplace =True)

#cross join tabla DesagregacionPronostico y calendario
df_calendario['key'] = 1
volumen_testing_ciudad['key'] = 1
DesagregacionPronosticoPlantaDia = pd.merge(df_calendario, volumen_testing_ciudad, on = 'key').drop("key",1)
DesagregacionPronosticoPlantaDia.rename(columns={'ciudad_asignaciones':'Ciudad'}, inplace = True)
DesagregacionPronosticoPlantaDia = DesagregacionPronosticoPlantaDia.drop(['ciudad_ic'], axis = 1)


#Consulta historico de volumen despachado diario por planta

despachosSQL2 = querySQL("{CALL SCAC_AP8_BaseForecast (?,?,?)}", 
                         (pais, 
                          inicioHistoria.strftime("%Y-%m-%d"), 
                          finHistoria.strftime("%Y-%m-%d") 
                         ) 
                        )

despachosSQL2['year_month'] = despachosSQL2.FechaEntrega.dt.to_period('M')
despachosSQL2['totalEntregado'] = despachosSQL2['totalEntregado'].astype(float)

#agrego informacion geografica        
nombre_cluster = querySQL( 
    "SELECT Pais, Centro, Ciudad_Cluster as Ciudad, [Desc Cluster] as Cluster, Plantas_fijas FROM SCAC_AT1_NombreCluster where Pais = ?" , 
    (pais) )

despachosSQL2 = pd.merge(
    despachosSQL2, 
    nombre_cluster, 
    left_on ='Planta', 
    right_on='Centro', 
    how = 'left'
)

#obtengo el listado de ciudades y plantas que han estado activas los ultimos N dias
DesagregacionPronosticoPlanta = despachosSQL2[
    despachosSQL2['FechaEntrega'] >= (finHistoria - datetime.timedelta(30)) - MonthBegin(1) 
]

DesagregacionPronosticoPlanta = DesagregacionPronosticoPlanta.groupby(
    ['Ciudad', 'Planta', 'Plantas_fijas']
).size().reset_index()

DesagregacionPronosticoPlanta = DesagregacionPronosticoPlanta.drop([0], axis = 1)

DesagregacionPronosticoPlantaDia = pd.merge(DesagregacionPronosticoPlantaDia, DesagregacionPronosticoPlanta, on='Ciudad' )


#Calculo de proporciones
proportion_week_1 = calculate_proportion_testing(despachosSQL2, 
                                         1, año_target, mes_target,
                                         ['Año', 'Mes', 'Planta'], 
                                         ['Año', 'Mes', 'Planta', 'Semana_Relativa'], 
                                         'totalEntregado', 
                                         ['Planta', 'Semana_Relativa'],
                                         'semana' + str(1))

proportion_week_2 = calculate_proportion_testing(despachosSQL2, 
                                         2, año_target, mes_target,
                                         ['Año', 'Mes', 'Planta'], 
                                         ['Año', 'Mes', 'Planta', 'Semana_Relativa'], 
                                         'totalEntregado', 
                                         ['Planta', 'Semana_Relativa'],
                                         'semana' + str(2))

proportion_week_3 = calculate_proportion_testing(despachosSQL2, 
                                         3, año_target, mes_target,
                                         ['Año', 'Mes', 'Planta'], 
                                         ['Año', 'Mes', 'Planta', 'Semana_Relativa'], 
                                         'totalEntregado', 
                                         ['Planta', 'Semana_Relativa'],
                                         'semana' + str(3))

proportion_week_6 = calculate_proportion_testing(despachosSQL2, 
                                         6, año_target, mes_target,
                                         ['Año', 'Mes', 'Planta'], 
                                         ['Año', 'Mes', 'Planta', 'Semana_Relativa'], 
                                         'totalEntregado', 
                                         ['Planta', 'Semana_Relativa'],
                                         'semana' + str(6))

proportion_weekday_1 = calculate_proportion_testing(despachosSQL2, 
                                         1, año_target, mes_target,
                                         ['Año', 'Mes', 'Planta'], 
                                         ['Año', 'Mes', 'Planta', 'DiaSemana'], 
                                         'totalEntregado', 
                                         ['Planta', 'DiaSemana'],
                                         'dia_semana' + str(1))

proportion_weekday_2 = calculate_proportion_testing(despachosSQL2, 
                                         2, año_target, mes_target,
                                         ['Año', 'Mes', 'Planta'], 
                                         ['Año', 'Mes', 'Planta', 'DiaSemana'], 
                                         'totalEntregado', 
                                         ['Planta', 'DiaSemana'],
                                         'dia_semana' + str(2))

proportion_weekday_3 = calculate_proportion_testing(despachosSQL2, 
                                         3, año_target, mes_target,
                                         ['Año', 'Mes', 'Planta'], 
                                         ['Año', 'Mes', 'Planta', 'DiaSemana'], 
                                         'totalEntregado', 
                                         ['Planta', 'DiaSemana'],
                                         'dia_semana' + str(3))

proportion_weekday_6 = calculate_proportion_testing(despachosSQL2, 
                                         6, año_target, mes_target,
                                         ['Año', 'Mes', 'Planta'], 
                                         ['Año', 'Mes', 'Planta', 'DiaSemana'], 
                                         'totalEntregado', 
                                         ['Planta', 'DiaSemana'],
                                         'dia_semana' + str(6))
# Medias diarias
media_diaria1 = media_diaria_testing(despachosSQL2, 1, año_target, mes_target)
media_diaria2 = media_diaria_testing(despachosSQL2, 2, año_target, mes_target)
media_diaria3 = media_diaria_testing(despachosSQL2, 3, año_target, mes_target)
media_diaria6 = media_diaria_testing(despachosSQL2, 6, año_target, mes_target)

#volumen_ciudad_mes = volumen_mes_testing(despachosSQL2, año_target, mes_target, finHistoria )

DesagregacionPronosticoPlantaDia.rename(
    columns={'Dia_Semana':'DiaSemana', 'Semana_relativa': 'Semana_Relativa', 'Plantas_fijas':'PlantaCentral'}, 
    inplace = True
)

datatesting = DesagregacionPronosticoPlantaDia[[
    'Año', 
    'Mes', 
    'Ciudad',
    'Planta',
    'Semana_Relativa', 
    'DiaSemana', 'PlantaCentral']]

datatesting['DiaSemana'] = datatesting['DiaSemana'].astype(int)

datatesting = pd.merge(datatesting, proportion_week_1, on=['Año', 'Mes', 'Planta', 'Semana_Relativa'], how='left')
datatesting = pd.merge(datatesting, proportion_week_2, on=['Año', 'Mes', 'Planta', 'Semana_Relativa'], how='left')
datatesting = pd.merge(datatesting, proportion_week_3, on=['Año', 'Mes', 'Planta', 'Semana_Relativa'], how='left')
datatesting = pd.merge(datatesting, proportion_week_6, on=['Año', 'Mes', 'Planta', 'Semana_Relativa'], how='left')

datatesting = pd.merge(datatesting, proportion_weekday_1, on=['Año', 'Mes', 'Planta', 'DiaSemana'], how='left')
datatesting = pd.merge(datatesting, proportion_weekday_2, on=['Año', 'Mes', 'Planta', 'DiaSemana'], how='left')
datatesting = pd.merge(datatesting, proportion_weekday_3, on=['Año', 'Mes', 'Planta', 'DiaSemana'], how='left')
datatesting = pd.merge(datatesting, proportion_weekday_6, on=['Año', 'Mes', 'Planta', 'DiaSemana'], how='left')

datatesting = pd.merge(datatesting, media_diaria1, on=['Año', 'Mes', 'Planta'], how='left')
datatesting = pd.merge(datatesting, media_diaria2, on=['Año', 'Mes', 'Planta'], how='left')
datatesting = pd.merge(datatesting, media_diaria3, on=['Año', 'Mes', 'Planta'], how='left')
datatesting = pd.merge(datatesting, media_diaria6, on=['Año', 'Mes', 'Planta'], how='left')

#datatesting= pd.merge(datatesting, volumen_ciudad_mes, on=['Año','Mes', 'Ciudad'], how='left')
datatesting['volumen_ciudad'] = volumen_mes_testing(despachosSQL2, año_target, mes_target, finHistoria )

datatesting['PlantaCentral'] = np.select(
    [
        datatesting['PlantaCentral'] == 'Central'
    ],
    [
        1
    ],default=0)

datatesting = datatesting.fillna(0)


columnas = ['Año', 'Mes', 'Planta','Semana_Relativa', 'DiaSemana', 
            'proportion_semana1', 'proportion_semana2',
            'proportion_semana3', 'proportion_semana6', 
            'proportion_dia_semana1','proportion_dia_semana2', 
            'proportion_dia_semana3', 'proportion_dia_semana6', 
            'media1', 'media2', 'media3', 'media6',
            'PlantaCentral', 'volumen_ciudad']

datatesting = datatesting[columnas]

datatesting.to_csv('../datos/df_datatesting.csv')

# Formato Final

In [91]:
result_consejo = pd.read_excel('../datos/resultado_consenso.xlsx')

# ajustes finales

df_calendario = querySQL( 
    "SELECT * FROM SCAC_AT3_DiasHabilesFuente WHERE pais = ? and [Fecha de entrega]  between ? and ? " , 
    (pais, datetime.datetime(año_target, mes_target, 1).strftime("%Y-%m-%d"), 
     (datetime.datetime(año_target, mes_target, 1) + MonthEnd(1)).strftime("%Y-%m-%d") ))

df_calendario.rename(
    columns={'Fecha de entrega':'FechaEntrega', 'Dia_Semana': 'DiaSemana', 'Semana_relativa': 'Semana_Relativa'}, 
    inplace =True)


#agrego informacion geografica        
nombre_cluster = querySQL( 
    "SELECT Pais, Centro, Ciudad_Cluster as Ciudad, [Desc Cluster] as Cluster, [Planta Unica] as PlantaUnica FROM SCAC_AT1_NombreCluster where Pais = ?" , 
    (pais) )

df_calendario.Año = df_calendario.Año.astype(str)
df_calendario.Mes = df_calendario.Mes.astype(str)
df_calendario.DiaSemana = df_calendario.DiaSemana.astype(str)
df_calendario.Semana_Relativa = df_calendario.Semana_Relativa.astype(str)

result_consejo['DiaSemana'] = result_consejo['DiaSemana'].astype(float).round(0)
result_consejo['DiaSemana'] = result_consejo['DiaSemana'].astype(int)
result_consejo['DiaSemana'] = result_consejo['DiaSemana'].astype(str)

result_consejo['Semana_Relativa'] = result_consejo['Semana_Relativa'].astype(float).round(0)
result_consejo['Semana_Relativa'] = result_consejo['Semana_Relativa'].astype(int)
result_consejo['Semana_Relativa'] = result_consejo['Semana_Relativa'].astype(str)

result_consejo['Año'] = result_consejo['Año'].astype(float).round(0)
result_consejo['Año'] = result_consejo['Año'].astype(int)
result_consejo['Año'] = result_consejo['Año'].astype(str)

result_consejo['Mes'] = result_consejo['Mes'].astype(float).round(0)
result_consejo['Mes'] = result_consejo['Mes'].astype(int)
result_consejo['Mes'] = result_consejo['Mes'].astype(str)

result_consejo = pd.merge(result_consejo, df_calendario[['Año', 'Mes', 'DiaSemana', 'Semana_Relativa', 'FechaEntrega']], on=['Año', 'Mes', 'DiaSemana', 'Semana_Relativa' ], how='inner')
result_consejo = pd.merge(result_consejo, nombre_cluster, left_on=['Planta'], right_on=['Centro'] )

#columnas_finales = ['Pais', 'Ciudad', 'Centro', 'PlantaUnica', 'FechaEntrega', 'PrediccionRNR', 'PrediccionRF',
#       'PrediccionXG', 'consenso_ia']

columnas_finales = ['Pais', 'Ciudad', 'Centro', 'PlantaUnica', 'FechaEntrega','consenso_ia']
result_consejo = result_consejo[columnas_finales]

In [92]:
result_consejo.to_excel("../datos/Desagregacion_" + pais + "_" + pd.to_datetime("now").strftime("%Y-%m-%d-%H-%M-%S") + ".xlsx", index = False)

In [93]:
"""
PARAMETROS:
absorcionEstadistica = 1  -> Toma proporciones historicas
absorcionEstadistica = 0  -> Toma proporciones por archivo de volumen CIUDAD
absorcionEstadistica = -1 -> Toma proporciones por archivo de volumen LINEA PRODUCTIVA

"""
absorcionEstadistica = 1 #criterio para tomar las proporciones historicas de las plantas o False -> archivo que define el volumen por planta
volPais = 140152

# si es estadistico tomo el resultado de la ia y con el volumen pais desagrego volumenes por ciudad
if absorcionEstadistica == 1:

    absorcion_ciudad = result_consejo.groupby(['Ciudad'])['consenso_ia'].sum().reset_index()
    absorcion_ciudad['volumen_pais_ia'] = volumen_pais
    absorcion_ciudad['absorcion_ciudad'] = absorcion_ciudad['consenso_ia'] / absorcion_ciudad['volumen_pais_ia']
    absorcion_ciudad['volumen_ciudad'] = absorcion_ciudad['absorcion_ciudad'] * volPais
    
elif absorcionEstadistica == 0:
    
    absorcion_ciudad = pd.read_excel('../DatosAbsorcionPlantas/ColombiaCiudad.xlsx')
    absorcion_ciudad.rename(columns={'ciudad_asignaciones':'Ciudad','volumen':'volumen_ciudad'}, inplace = True)

absorcion_ciudad = absorcion_ciudad[['Ciudad', 'volumen_ciudad']]
absorcion_ciudad

,Ciudad,volumen_ciudad
0,Barranquilla,8840.559770
1,Bogotá,69365.409525
2,Bucaramanga,10434.383970
3,Cali,11370.342292
4,Cartagena,5623.813258
5,Cúcuta,1990.385566
6,Fusagasuga,2564.305497
7,Ibagué,8728.999885
8,Maceo,447.851479
9,Medellín,9283.152332


In [67]:
absorcion_centro_aux = result_consejo.groupby(['Ciudad'])['consenso_ia'].sum().reset_index()
absorcion_centro_aux.rename(columns={'consenso_ia':'volumen_ciudad'}, inplace = True)
absorcion_centro = result_consejo.groupby(['Ciudad', 'Centro'])['consenso_ia'].sum().reset_index()
absorcion_centro = pd.merge(absorcion_centro, absorcion_centro_aux, on ='Ciudad')
absorcion_centro['absorcion_centro'] = absorcion_centro['consenso_ia'] / absorcion_centro['volumen_ciudad']
absorcion_centro = absorcion_centro[['Ciudad','Centro', 'absorcion_centro']]

absorcion_centro = pd.merge(absorcion_centro, absorcion_ciudad, on ='Ciudad')
absorcion_centro['volumen_centro'] = absorcion_centro['absorcion_centro'] * absorcion_centro['volumen_ciudad']

absorcion_centro = absorcion_centro[['Centro', 'volumen_centro']]
absorcion_centro

,Centro,volumen_centro
0,F014,6555.802540
1,F001,6510.671471
2,F003,3747.322003
3,F006,3858.750712
4,F007,6383.075391
5,F030,2830.938929
6,F048,4624.504506
7,F058,5600.707537
8,F069,5514.072834
9,F080,6072.320348


In [68]:
absorcion_dia_centro_aux = result_consejo.groupby(['Centro'])['consenso_ia'].sum().reset_index()
absorcion_dia_centro_aux.rename(columns={'consenso_ia':'volumen_centro'}, inplace = True)
absorcion_dia_centro = result_consejo.groupby(['Centro','FechaEntrega'])['consenso_ia'].sum().reset_index() # este paso depronto sobra
absorcion_dia_centro = pd.merge(absorcion_dia_centro, absorcion_dia_centro_aux, on ='Centro')
absorcion_dia_centro['absorcion_dia'] = absorcion_dia_centro['consenso_ia'] / absorcion_dia_centro['volumen_centro']

absorcion_dia_centro = absorcion_dia_centro[['Centro', 'FechaEntrega', 'absorcion_dia']]
absorcion_dia_centro

,Centro,FechaEntrega,absorcion_dia
0,F001,2022-06-01,0.041531
1,F001,2022-06-02,0.043798
2,F001,2022-06-03,0.047778
3,F001,2022-06-04,0.033920
4,F001,2022-06-06,0.039902
...,...,...,...
739,FB89,2022-06-24,0.042184
740,FB89,2022-06-25,0.038283
741,FB89,2022-06-28,0.042581
742,FB89,2022-06-29,0.043419


In [69]:
result_consejo.

,Pais,Ciudad,Centro,PlantaUnica,FechaEntrega,consenso_ia
0,Colombia,Bogotá,F001,CO-PLANTA 240,2022-06-01,360.700000
1,Colombia,Bogotá,F001,CO-PLANTA 240,2022-06-02,380.390000
2,Colombia,Bogotá,F001,CO-PLANTA 240,2022-06-03,414.954167
3,Colombia,Bogotá,F001,CO-PLANTA 240,2022-06-04,294.597083
4,Colombia,Bogotá,F001,CO-PLANTA 240,2022-06-06,346.554167
...,...,...,...,...,...,...
739,Colombia,Maceo,FA05,CO-PLANTA MACEO,2022-06-24,29.289583
740,Colombia,Maceo,FA05,CO-PLANTA MACEO,2022-06-25,30.888842
741,Colombia,Maceo,FA05,CO-PLANTA MACEO,2022-06-28,29.812500
742,Colombia,Maceo,FA05,CO-PLANTA MACEO,2022-06-29,31.582018


In [70]:
result_consejo_ajustado = pd.merge(result_consejo, absorcion_centro, on ='Centro')
result_consejo_ajustado = pd.merge(result_consejo_ajustado, absorcion_dia_centro, on =['Centro', 'FechaEntrega'])
result_consejo_ajustado['M3Forecast'] = result_consejo_ajustado['volumen_centro'] * result_consejo_ajustado['absorcion_dia']
result_consejo_ajustado

,Pais,Ciudad,Centro,PlantaUnica,FechaEntrega,consenso_ia,volumen_centro,absorcion_dia,M3Forecast
0,Colombia,Bogotá,F001,CO-PLANTA 240,2022-06-01,360.700000,6510.671471,0.041531,270.393655
1,Colombia,Bogotá,F001,CO-PLANTA 240,2022-06-02,380.390000,6510.671471,0.043798,285.153985
2,Colombia,Bogotá,F001,CO-PLANTA 240,2022-06-03,414.954167,6510.671471,0.047778,311.064524
3,Colombia,Bogotá,F001,CO-PLANTA 240,2022-06-04,294.597083,6510.671471,0.033920,220.840538
4,Colombia,Bogotá,F001,CO-PLANTA 240,2022-06-06,346.554167,6510.671471,0.039902,259.789431
...,...,...,...,...,...,...,...,...,...
739,Colombia,Maceo,FA05,CO-PLANTA MACEO,2022-06-24,29.289583,509.438581,0.043099,21.956522
740,Colombia,Maceo,FA05,CO-PLANTA MACEO,2022-06-25,30.888842,509.438581,0.045453,23.155383
741,Colombia,Maceo,FA05,CO-PLANTA MACEO,2022-06-28,29.812500,509.438581,0.043869,22.348519
742,Colombia,Maceo,FA05,CO-PLANTA MACEO,2022-06-29,31.582018,509.438581,0.046473,23.675013


In [78]:
result_consejo_ajustado[result_consejo_ajustado['Centro']== 'F030']

,Pais,Ciudad,Centro,PlantaUnica,FechaEntrega,consenso_ia,volumen_centro,absorcion_dia,M3Forecast
96,Colombia,Bogotá,F030,CO-PLANTA SOACHA,2022-06-01,150.156250,2830.938929,0.039762,112.562510
97,Colombia,Bogotá,F030,CO-PLANTA SOACHA,2022-06-02,151.906250,2830.938929,0.040225,113.874372
98,Colombia,Bogotá,F030,CO-PLANTA SOACHA,2022-06-03,167.391667,2830.938929,0.044326,125.482796
99,Colombia,Bogotá,F030,CO-PLANTA SOACHA,2022-06-04,139.239583,2830.938929,0.036871,104.378985
100,Colombia,Bogotá,F030,CO-PLANTA SOACHA,2022-06-06,143.968048,2830.938929,0.038123,107.923612
101,Colombia,Bogotá,F030,CO-PLANTA SOACHA,2022-06-07,160.389583,2830.938929,0.042471,120.233783
102,Colombia,Bogotá,F030,CO-PLANTA SOACHA,2022-06-08,155.479019,2830.938929,0.041171,116.552648
103,Colombia,Bogotá,F030,CO-PLANTA SOACHA,2022-06-09,161.160417,2830.938929,0.042675,120.811628
104,Colombia,Bogotá,F030,CO-PLANTA SOACHA,2022-06-10,164.002083,2830.938929,0.043428,122.941843
105,Colombia,Bogotá,F030,CO-PLANTA SOACHA,2022-06-11,144.166667,2830.938929,0.038175,108.072503


In [75]:
result_consejo_ajustado['consenso_ia'].sum()

146243.28776539103